In [2]:
from core.environment import get_system_language,get_current_time,get_user_coordinates,is_valid_coordinates
from i18n import t
from core.llm import get_model

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END


from tools.external_apis import get_weather,get_location

from typing_extensions import List, TypedDict
from schemas import PoemState
from pydantic import BaseModel, Field

# 语言和时间
language_code = get_system_language()
time_string = get_current_time()

# 定义返回结构
class PoemOutPut(BaseModel):
    author: str = Field(description=t(language_code,"author"))
    title: str = Field(
        description=t(language_code,"title_of_poem")
    )
    content: str = Field(description=t(language_code,"line_of_poem"))
    wholePoem: str = Field(description=t(language_code,"entire_poem"))
    reason: str = Field(description=t(language_code,"reason"))
    
class TimeFormatOutPut(BaseModel):
    season: str = Field(
        json_schema_extra={"enum": t(language_code,"season")}
    )
    timeBlock: str = Field(
        json_schema_extra={"enum": t(language_code,"time")}
    )

# 收集 Location
while True:
    lat_str, lon_str = get_user_coordinates()
    is_valid, lat_val, lon_val = is_valid_coordinates(lat_str, lon_str)
    if is_valid:
        break
    else:
        print(t(language_code,"geo_wrong"))

# 选择模型t
model = get_model(provide='lmstudio',model_name='qwen/qwen3-next-80b')

def time_node (state: PoemState):
    parser = JsonOutputParser(pydantic_object=TimeFormatOutPut)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "{system_prompt}\n\n{format_instructions}"),
        ("user", "{user_input}")
    ]).partial(
        system_prompt=t(language_code,"reply"),
        user_input=f"{t(language_code,'generate_season')}:{time_string}",
        format_instructions=parser.get_format_instructions()
    )
    chain = prompt | model | parser
    result = chain.invoke({})
    print (f"{result['season']} {result['timeBlock']}")
    return {"time_info": f"{result['season']} {result['timeBlock']}"}

async def weather_node (state: PoemState):
    result = await get_weather(lat_val,lon_val,language_code)
    if result['ok']:

        def deg_to_cardinal(deg):
            dirs = t(language_code, "dirs")
            idx = round(deg / 45) % 8
            return dirs[idx]

        wind_dir_text = deg_to_cardinal(result["data"]["wind_direction"]) if result["data"]["wind_direction"] is not None else "Unknown"
        wind_str = f"{result['data']['wind_speed']} km/h，{wind_dir_text}" if result["data"]["wind_speed"] is not None and result["data"]["wind_direction"] is not None else "Unknown"
        weather_text = t(language_code, "weather_descriptions").get(result["data"]["weather_code"], "Unknown") if result["data"]["weather_code"] is not None else "Unknown"   
        temp_str = f"{result['data']['temp']}℃" if result["data"]["temp"] is not None else "Unknown"
        humidity_str = f"{result['data']['humidity']}%" if result["data"]["humidity"] is not None else "Unknown"
        
        text = t(language_code,"weather_now",weather_text=weather_text,temp_str=temp_str,humidity_str=humidity_str,wind_str=wind_str)
    else:
        text = t(language_code,"query_failed",status_code=result["error"])
    print(text)    
    return {"weather_info": text}

async def location_node (state: PoemState):
    result = await get_location(lat_val,lon_val,language_code)
    if result['ok']:
        text = t(language_code,"location_is",location_str=result["data"])
    else:
        text = t(language_code,"query_failed",status_code=result["error"])
    print(text)    
    return {"location_info": text}

def culture_node (state: PoemState):
    loc = state["location_info"]
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "{system_prompt}"),
        ("user", "{user_input}")
    ]).partial(
        system_prompt=t(language_code,"reply"),
        user_input=f"{loc}，{t(language_code,'summarize_cultural')}"
    )
    chain = prompt | model
    result = chain.invoke({})
    print(result.content)
    return {"culture_info": result.content}

def poem_node (state: PoemState):
    parser = JsonOutputParser(pydantic_object=PoemOutPut)

    context = f"""
        {state.get('location_info')}，
        {state.get('time_info')}。
        {state.get('weather_info')}。\n\n
        {state.get('culture_info')} \n\n
        {t(language_code,'find_poem')}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "{system_prompt}\n\n{format_instructions}"),
        ("user", "{user_input}")
    ]).partial(
        system_prompt=t(language_code,"reply"),
        user_input=context,
        format_instructions=parser.get_format_instructions()
    )
    chain = prompt | model | parser
    result = chain.invoke({})
    print(t(language_code,"final_demonstrate",content=result['content'],author=result['author'],reason=result['reason'],wholePoem=result['wholePoem']))
    return {"poem": result}    
    
builder = StateGraph(PoemState)
builder.add_node("time", time_node)
builder.add_node("weather", weather_node)
builder.add_node("location", location_node)
builder.add_node("culture", culture_node)
builder.add_node("poem", poem_node)

builder.add_edge(START, "time")
builder.add_edge(START, "weather")
builder.add_edge(START, "location")

builder.add_edge("location", "culture")

builder.add_edge(["culture", "time", "weather"], "poem")

builder.add_edge("poem", END)
graph = builder.compile()

result = await graph.ainvoke({})


请输入纬度（-90 ~ 90）： 40.43
请输入经度（-180 ~ 180）： 74


My current location is：Osh Region, Kyrgyzstan
Late Winter/Early Spring Evening
Current weather:Clear sky；Temperature：-0.9℃；Humidity：52%；Wind speed:2.9 km/h，West wind
Osh Region, located in southern Kyrgyzstan, is one of the country’s most historically significant areas, home to the ancient city of Osh—believed to be one of Central Asia’s oldest continuously inhabited cities—and a vital hub along the Silk Road, known for its rich blend of Kyrgyz and Uzbek cultures, vibrant bazaars, and the sacred Mount Sulayman, a UNESCO World Heritage site.


Recommended poem for you：The woods are lovely, dark and deep.

Author：Robert Frost

Reason for recommendation：The poem captures a quiet, contemplative winter evening with a clear sky and stillness, matching the serene atmosphere of Osh Region in late winter/early spring. The imagery of snow-covered woods and calm winds aligns with the described weather and tranquil setting.

The whole poem ：

Whose woods these are I think I know.
His house is in t